In [ ]:
# Import necessary libraries for the project
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from collections import Counter
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [ ]:
# Download required NLTK resources for tokenization and stop words
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
warnings.filterwarnings('ignore')

### --- Exploratory Data Analysis (EDA) ---

**Purpose:** Understand the dataset structure and identify patterns impacting classification.

**Why:** Analyzing class distribution, numerical values, and text terms helps detect imbalances or discriminative features.


In [ ]:
# Load the dataset
df_inv = pd.read_excel(
    '/kaggle/input/accounting-classification-suppliers-purchases/invoices_classification.xlsx',
    engine='openpyxl'
)

# Display basic dataset information
print("Dataset Information:")
print(df_inv.info())

In [ ]:
# Display first five rows
print("\nFirst 5 Rows:")
print(df_inv.head())

In [ ]:
# Display last five rows
print("\nLast 5 Rows:")
print(df_inv.tail())

> We can see that the dataset is already split into training and testing sets by the collumn 'Dataset'.

In [ ]:
# Plot 1: Distribution of Account_Number
plt.figure(figsize=(12, 6))
sns.countplot(data=df_inv, x='Account_Number', order=df_inv['Account_Number'].value_counts().index)
plt.title('Distribution of Classes (Account_Number)')
plt.xlabel('Account Number')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


> There are many classes with few registers.

In [ ]:
# Create contingency table
contingency_table = pd.crosstab(df_inv['Supplier_Code'], df_inv['Account_Number'])

# Calculate Cramér's V
def cramers_v(contingency_table):
    """
    Purpose: Compute Cramér's V to measure association between two categorical variables.
    """
    chi2, p, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    return np.sqrt(chi2 / (n * min_dim))

cramers_v_value = cramers_v(contingency_table)
print(f"\nCramér's V between {'Supplier_Code'} and Account_Number: {cramers_v_value:.3f}")
print(f"Chi-square p-value: {chi2_contingency(contingency_table)[1]:.3f}")
# Why: Cramér's V (0 to 1) indicates association strength; a low p-value (<0.05) suggests a significant relationship.

> The **chi-square** test evaluates whether the relationship between Supplier_Code and Account_Number is statistically significant. The p-value indicates the probability that the observed association occurred by chance.
> 
> 
> 
> **Cramér's V = 0.842** indicates that suppliers are a key differentiator for accounting accounts.
> 
> 
> 
> A **p-value of 0.000** (typically < 0.05) means the relationship is statistically significant.

In [ ]:
# Plot 2: Heatmap of contingency table
plt.figure(figsize=(12, 8))
sns.heatmap(contingency_table, annot=True, fmt='d', cmap='Blues')
plt.title(f'Contingency Table: Supplier_Code vs. Account_Number')
plt.xlabel('Account Number')
plt.ylabel('Supplier_Code')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

> The heatmap allows us to visualize the distribution of supplier-account pairs, emphasizing patterns of strong association.
> 
> 
> 
> We observe that most suppliers are linked to a single account, with only a few having items classified across multiple accounts


### --- Data Preprocessing ---

In [ ]:
# Split the dataset into training and test sets.

df_inv_train = df_inv[df_inv["Dataset"] == "Training"]
df_inv_test = df_inv[df_inv["Dataset"] == "Test"]

In [ ]:
# Create features and handle missing values to prepare the dataset for modeling.

X_train = df_inv_train["Supplier_Name"].fillna("") + " " + df_inv_train["Item_Description"].fillna("")
y_train = df_inv_train["Account_Number"]

X_test = df_inv_test["Supplier_Name"].fillna("") + " " + df_inv_test["Item_Description"].fillna("")
y_test = df_inv_test["Account_Number"]

In [ ]:
# Defines a dictionary of three text vectorization methods 
# (TfidfVectorizer, CountVectorizer, and HashingVectorizer) and evaluates their performance

vectorizers = {
    "TF-IDF": TfidfVectorizer(ngram_range=(1,2)),
    "Count": CountVectorizer(ngram_range=(1,2)),
    "Hashing": HashingVectorizer(n_features=5000, alternate_sign=False)
}

results = {}

for name, vec in vectorizers.items():
    pipe = Pipeline([
        ("vec", vec),
        ("clf", LogisticRegression(max_iter=1000))
    ])
    scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring="f1_macro")
    results[name] = scores.mean()

results

Text vectorization converts textual data (e.g., "Cleaning products (kit)") into numerical features that a machine learning model like logistic regression can process. Different vectorizers capture different aspects of the text:

**TfidfVectorizer:** Emphasizes rare but important terms by weighting them higher (Term Frequency-Inverse Document Frequency).

**CountVectorizer:** Counts the frequency of words or n-grams, treating all terms equally.

**HashingVectorizer:** Maps terms to a fixed-size feature space using a hash function, useful for large datasets but less interpretable.

Choosing the right vectorizer is critical for model performance, especially since the dataset includes Item_Description with discriminative terms.

In [ ]:
# Build a pipeline for preprocessing and modeling.

pipeline = Pipeline([
    ("count", CountVectorizer(ngram_range=(1,2))),
    ("clf", LogisticRegression(max_iter=1000))
])
pipeline.fit(X_train, y_train)

> A pipeline automates data transformation (text and numerical) and model application, ensuring consistency.

In [ ]:
# Assess the model’s performance on the test set, providing metrics

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

**Accuracy** = 0.92 (92%): The model correctly classified 92% of the 51 test instances.

**Macro Average** (Precision = 0.83, Recall = 0.86, F1 = 0.83): These metrics average performance across all classes equally, regardless of class frequency. The macro F1-score of 0.83 indicates good performance across classes.

**Weighted Average** (Precision = 0.91, Recall = 0.92, F1 = 0.91): These metrics weight each class by its frequency in the test set. The higher weighted F1 (0.91 vs. 0.83 macro) suggests the model performs better on frequent classes but may struggle with rare ones.

**Sample Size** = 51: The test set has 51 instances, which is relatively small, so performance metrics may be sensitive to a few misclassifications.

In [ ]:
# Create a DataFrame with actual and predicted labels
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Count correct predictions per class
accuracy_by_class = results_df.groupby('Actual').apply(
    lambda x: (x['Actual'] == x['Predicted']).sum()
).rename('Correct')

# Count total samples per class
total_by_class = results_df['Actual'].value_counts().rename('Total')

# Combine the two
accuracy_df = pd.concat([accuracy_by_class, total_by_class], axis=1)
accuracy_df['Accuracy'] = accuracy_df['Correct'] / accuracy_df['Total']

# Sort from lowest to highest accuracy
accuracy_df = accuracy_df.sort_values(by='Accuracy')

# Plot
plt.figure(figsize=(12, 6))
sns.barplot(x=accuracy_df.index, y=accuracy_df['Accuracy'])
plt.title('Accuracy by Account_Number)')
plt.ylabel('Accuracy (%)')
plt.xticks(rotation=45, ha='right')
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Filter accounts with accuracy less than 1.0
accounts_with_errors = accuracy_df[accuracy_df['Accuracy'] < 1.0].index

# Filter df_inv for these accounts
df_with_errors = df_inv[df_inv['Account_Number'].isin(accounts_with_errors)]

# Display
df_with_errors.head(100)

In [ ]:
# Filter only the rows with classification errors
errors_df = results_df[results_df['Actual'] != results_df['Predicted']]

# Display the errors
print(errors_df)

In [ ]:
# Grup by Account_Number e calculates average Item_Value
avg_value = df_inv.groupby('Account_Number')['Item_Value'].mean()

# Plot
plt.figure(figsize=(12, 6))
avg_value.plot(kind='bar', color='skyblue')
plt.title('Averege Value by Account Number')
plt.ylabel('Averege Value (Item_Value)')
plt.xlabel('Account Number')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

> **Account 1001004:** This account exhibits a slightly higher average Item_Value compared to accounts 5000024 and 5000034. This difference likely stems from thresholds set by company policies or regulations for classifying an asset as a fixed asset (immobilized) versus a low-value expense. Including Item_Value as a feature could improve the model’s ability to correctly classify this account by capturing these value-based distinctions.
> 
> 
> 
> **Account 5000060:** The model predicted account 4000060 instead of 5000060. The primary distinguishing information between accounts in group 4 (e.g., 4000060) and group 5 (e.g., 5000060) lies in the Item_Description field, specifically the presence of 'B' in "Monthly rent – Warehouse B" for group 5 versus 'A' in "Monthly rent – Warehouse A" for group 4. As the rental prices for Warehouse A and Warehouse B differ, incorporating Item_Value as a feature may enhance the model’s ability to differentiate these accounts, potentially improving accuracy.
> 
> 
> 
> **Account 4000006:** The supplier CSF Wholesale and Retail and the item Cleaning products (kit) are associated with both accounts 4000006 and 4000013 due to a misclassification in the dataset. It will be necessary to assign the correct account to improve the model's predictive performance.

In [ ]:
# Fix the Account_Number for invoice 793 to align with invoice 491.
df_inv.loc[(df_inv['Invoice'] == 793) &
           (df_inv['Supplier_Name'] == 'CSF Wholesale and Retail') &
           (df_inv['Item_Description'] == 'Cleaning products (kit)'),
           'Account_Number'] = '4000013'

# Verify the correction
print("\nVerification of correction for invoice 793:")
print(df_inv[(df_inv['Invoice'] == 793) &
             (df_inv['Supplier_Name'] == 'CSF Wholesale and Retail')]
             [['Invoice', 'Supplier_Name', 'Item_Description', 'Account_Number']])

In [ ]:
# Prefix the supplier name with something like "SUPPLIER_", which helps the model learn that these are 
# tokens distinct from those in the description, while still keeping everything in the same text field
df_inv['Input_Text'] = df_inv['Item_Description'] + ' SUPPLIER_' + df_inv['Supplier_Name']

# Create Log_Item_Value
df_inv['Log_Item_Value'] = np.log1p(df_inv['Item_Value'])


In [ ]:
# Split the dataset into training and test sets.
df_inv_train = df_inv[df_inv["Dataset"] == "Training"]
df_inv_test = df_inv[df_inv["Dataset"] == "Test"]


In [ ]:
# Define features and target
X_train = df_inv_train[['Input_Text', 'Item_Value', 'Log_Item_Value']]
y_train = df_inv_train['Account_Number'].astype(str)  # Ensure string
X_test = df_inv_test[['Input_Text', 'Item_Value', 'Log_Item_Value']]
y_test = df_inv_test['Account_Number'].astype(str)

# Define pipeline
preprocessing = ColumnTransformer([
    ('text', CountVectorizer(ngram_range=(1, 2), max_features=3000,
                             stop_words=['and', 'the', 'a', 'of', 'with', 'for', 'in', 'on', 'it', 'from', 'to']), 'Input_Text'),
    ('value', StandardScaler(), ['Item_Value', 'Log_Item_Value'])
])
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

**Log_Item_Value** is created by applying a logarithmic transformation to Item_Value. Handles Skewed Distributions: Invoice amounts (Item_Value) are often right-skewed, with many small values and a few large outliers (e.g., 100 vs. 100000). The log transformation compresses the range, reducing the impact of extreme values and making the distribution more normal-like, which improves LogisticRegression’s performance.

**StandardScaler** standardizes Item_Value and Log_Item_Value by subtracting the mean and dividing by the standard deviation, resulting in features with a mean of 0 and a standard deviation of 1. CountVectorizer produces sparse, non-negative integer counts (e.g., term frequencies from Input_Text), while Item_Value (e.g., 500, 100000) and Log_Item_Value (e.g., 6.2, 11.5) have different scales. StandardScaler ensures all features contribute equally to LogisticRegression’s optimization, preventing large Item_Value magnitudes from dominating the model.

The **stop_words** are frequent but non-discriminative terms that appear across many invoices (e.g., “and” in “Cleaning products and supplies”). Excluding them focuses CountVectorizer on meaningful tokens (e.g., “Cleaning”, “SUPPLIER_CSF”), improving the signal-to-noise ratio and model performance.

In [ ]:
# Fit and evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("Test accuracy:", pipeline.score(X_test, y_test))
print("\nClassification report:")
print(classification_report(y_test, y_pred))

> The earlier pipeline (92% accuracy, macro F1 = 0.83) used only Input_Text with CountVectorizer. Adding Item_Value and Log_Item_Value (standardized) provided numerical context, addressing misclassifications (e.g., 5000060, 1001004) and boosting accuracy to 94%.
> 
> 
> 
> Balanced Classes: The class_weight='balanced' setting in LogisticRegression improved performance on rare classes, reflected in the high macro F1 (0.91).
> 
> 
> 
> Small Test Set: With only 51 test instances, reducing errors from 4 (92%) to 3 (94%) is significant

In [ ]:
# Create derived features
df_inv['Log_Taxes_Credit_Amount'] = np.log1p(df_inv['Taxes_Credit_Amount'])
df_inv['Value_Category'] = pd.cut(df_inv['Item_Value'], bins=[0, 500, 2000, 5000, 10000, float('inf')], 
                                 labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
df_inv['Value_Category'] = df_inv['Value_Category'].cat.add_categories('Unknown').fillna('Unknown')
# Purpose: Apply logarithmic transformation to Taxes_Credit_Amount to handle skewed distributions, 
# and create a categorical Value_Category feature based on Item_Value bins to capture value-based patterns.

In [ ]:
# Split dataset into training and test sets
df_inv_train = df_inv[df_inv["Dataset"] == "Training"]
df_inv_test = df_inv[df_inv["Dataset"] == "Test"]

# Define features and target
X_train = df_inv_train[['Input_Text', 'Log_Item_Value', 'Item_Quantity', 'Taxes_Credit_%', 'Log_Taxes_Credit_Amount', 'Value_Category']]
y_train = df_inv_train['Account_Number'].astype(str)  # Ensure string
X_test = df_inv_test[['Input_Text', 'Log_Item_Value', 'Item_Quantity', 'Taxes_Credit_%', 'Log_Taxes_Credit_Amount', 'Value_Category']]
y_test = df_inv_test['Account_Number'].astype(str)

In [ ]:
# Create a new pipeline that
preprocessing = ColumnTransformer([
    ('text', CountVectorizer(ngram_range=(1, 2), max_features=3000, 
                             stop_words=['and', 'the', 'a', 'of', 'with', 'for', 'in', 'on', 'it', 'from', 'to'], 
                             ), 'Input_Text'),
    ('value', StandardScaler(), ['Log_Item_Value']),
    ('quantity', StandardScaler(), ['Item_Quantity']),
    ('tax_rate', StandardScaler(), ['Taxes_Credit_%']),
    ('tax_credit', StandardScaler(), ['Log_Taxes_Credit_Amount']),
    ('category', CountVectorizer(max_features=10), 'Value_Category')
])
pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

In [ ]:
# Hyperparameter tuning
param_grid = {
    'preprocessing__text__max_features': [2000, 3000, 5000],
    'clf__C': [0.01, 0.1, 1, 10]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(n_splits=3), scoring='f1_macro')
grid_search.fit(X_train, y_train)
print("Best configuration:", grid_search.best_params_)
print("Best macro F1-score:", grid_search.best_score_)
pipeline = grid_search.best_estimator_
# Purpose: Perform hyperparameter tuning using GridSearchCV with 3-fold stratified cross-validation, 
# optimizing the number of text features (max_features) and LogisticRegression regularization strength
# (C) to maximize macro F1-score.

In [ ]:
# Test evaluation
y_pred = pipeline.predict(X_test)
print("Acurácia no teste:", pipeline.score(X_test, y_test))
print(classification_report(y_test, y_pred))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=pipeline.classes_)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=pipeline.classes_, yticklabels=pipeline.classes_)
plt.title('Confusion Matrix')
plt.ylabel('Real Class')
plt.xlabel('Predicted Class')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Wrong predicted classes 
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
erros_df = results_df[results_df['Actual'] != results_df['Predicted']]
erros_df = erros_df.merge(df_inv_test[['Supplier_Name', 'Item_Description', 'Item_Value', 'Item_Quantity', 'Taxes_Credit_%']], left_index=True, right_index=True)
print("Wrong predicted classes:")
print(erros_df)

> The model correctly classified 98% of the 51 test instances (approximately 50 correct, 1 incorrect). This matches your project’s target accuracy of 98%, a significant improvement from the earlier 94% (3 errors) and 92% (text-only pipeline).
> 
> 
> 
> > **Value_Category: **Binning Item_Value into categories (Very Low to Very High) captured ordinal patterns, improving classification for value-sensitive accounts like 1001004.
> 
> > 
> 
> > 
> 
> > 
> 
> > **Hyperparameter Tuning: **GridSearchCV optimized max_features and C, fine-tuning the text feature space and regularization, which likely reduced the error rate to 1.